In [151]:
import numpy as np
import igraph as ig
import geopandas as gpd
import pandas as pd
import math
from typing import Tuple, List
from sklearn.neighbors import KernelDensity
from matplotlib import pyplot as plt

In [152]:
g = ig.load('/home/rico/Documents/thesis/eda/notebooks/synthetic_dataset/graph.gml')
census = gpd.read_file('/home/rico/Documents/thesis/eda/notebooks/synthetic_dataset/census_data.geojson')

# Helper Functions

In [153]:
g.vs.select(type_eq='res_node')['name']

['RC1', 'RC2', 'RC3', 'RC4', 'RC5', 'RC6']

In [154]:
def evaluate_graph(g: ig.Graph) -> pd.DataFrame:
    nb_nodes = g.vs.select(type_eq='res_node')
    poi_nodes = g.vs.select(type_eq='poi_node')

    tt_mx = np.zeros((len(nb_nodes), len(poi_nodes)))
    hops_mx = np.zeros((len(nb_nodes), len(poi_nodes)))

    failed = {}

    for i, o in enumerate(nb_nodes):
        for j, d in enumerate(poi_nodes):
            # Travel Time
            tt = g.shortest_paths(o, d, weights='tt')[0][0]
            if tt == math.inf:
                if not failed.get(f"{o['node_id']}_tt", None) == d['node_id']:
                    failed[f"{o['node_id']}_tt"] = d["node_id"]
            else:
                tt_mx[i, j] = tt
            # Number of hops
            edges = g.get_shortest_paths(o, d, weights='tt', output='epath')[0]
            if edges == math.inf or not edges:
                if not failed.get(f"{o['node_id']}_edges", None) == d["node_id"]:
                    failed[f"{o['node_id']}_edges"] = d["node_id"]
            else:
                hops_mx[i, j] = len(edges)

    df_tt = pd.DataFrame(tt_mx, columns=poi_nodes['name'])
    df_tt['metric'] = 'travel_time'
    df_tt['rc'] = nb_nodes['name']

    df_hops = pd.DataFrame(hops_mx, columns=poi_nodes['name'])
    df_hops['metric'] = 'hops'
    df_hops['rc'] = nb_nodes['name']

    return pd.concat([df_tt,df_hops], axis=0)

In [155]:
evaluate_graph(g)

,POI1,POI2,POI3,metric,rc
0,53.413481,57.314920,53.413481,travel_time,RC1
1,44.598206,49.203658,44.598206,travel_time,RC2
2,53.413481,57.314920,53.413481,travel_time,RC3
3,33.541020,39.458839,33.541020,travel_time,RC4
4,33.541020,39.458839,33.541020,travel_time,RC5
5,37.589892,42.953463,37.589892,travel_time,RC6
0,1.000000,1.000000,1.000000,hops,RC1
1,1.000000,1.000000,1.000000,hops,RC2
2,1.000000,1.000000,1.000000,hops,RC3
3,1.000000,1.000000,1.000000,hops,RC4


In [156]:
def generate_samples(metric_matrix: np.array, census: gpd.GeoDataFrame):
    pass

# Reward Functions

In [157]:
def egalitarian(g: ig.Graph, census_data: gpd.GeoDataFrame) -> float:
    # fit KDE (sklearn) on each component
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(X)
    # fit KDE on all aggregated data (mixture)

    # kde_mixture.entropy - 1/n_components * sum([kde_component.entropy  for  kde_component in components] )


In [158]:
def utilitarian() -> float:
    pass

In [159]:
def elitarian() -> float:
    pass

# Random Baseline

# Fully Informed Greedy Baseline

# Genetic Algorithm Baseline

# Q-Learning